# Spark SQL

In [ ]:
spark.sparkContext.setLogLevel("ERROR")

### Check all the `databases` present

In [ ]:
spark.sql('SHOW databases').show()           # This is coming from AWS Glue Catalog

### Check the present `database` selected

In [ ]:
spark.catalog.currentDatabase() # This with show the presently selected `database`

### Check all the `tables` present

In [ ]:
spark.sql('SHOW tables').show() # In the `default` Database

### Select a particular `database`

In [ ]:
spark.sql('USE db_youtube_raw')   # Select a differet database

In [ ]:
spark.catalog.currentDatabase()

### Show the `tables` within the `database`

In [ ]:
spark.sql('SHOW tables').show(truncate=False)  # Show all the tables within the database 

### Create a database

In [ ]:
spark.sql('CREATE DATABASE IF NOT EXISTS my_db_spark')

In [ ]:
spark.sql('SHOW databases').show()

In [ ]:
spark.sql('USE my_db_spark')              # Use this newly created database 

In [ ]:
spark.sql('SHOW tables').show()           # Show all the tables within the database 

In [ ]:
# Check if a particular database is present or not (in case the list is long)

( 
    spark
    .sql('SHOW databases')
    .filter("namespace = 'db_youtube_raw'")
    .show() 
)

In [ ]:
# Another way to check if a list of databases are present or not (in case the list is long)

( 
    spark
    .sql('SHOW databases')
    .filter("namespace LIKE 'db%'")
    .show() 
)

### Create a Table

In [ ]:
spark.sql('USE my_db_spark') 
spark.catalog.currentDatabase()    # Check the present database (which is selected)

In [ ]:
spark.sql('SHOW tables').show(truncate=False)           

In [ ]:
spark.sql('CREATE TABLE orders \
               (order_id integer, \
                order_date string, \
                customer_id integer, \
                order_status string)')

In [ ]:
spark.sql('SHOW tables').show()

> Now you can go to a new shell and run this 
```python
spark.sql('USE my_db_spark')
spark.sql('SHOW tables').show()
+-----------+---------+-----------+                                             
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|my_db_spark|   orders|      false|
+-----------+---------+-----------+
```

### Create a `TempView` using the DataFrame

In [ ]:
# Lets first create a DF using that we will create a TempView
dataset = 's3://fcc-spark-example/dataset/2023/orders.csv'
df = spark.read.csv(dataset, header=True, inferSchema=True)

# Creating a TempView
df.createOrReplaceTempView('orders_view')

In [ ]:
spark.sql('SHOW tables').show()

> Now you can go to a new shell and run this **(WE WON'T SEE THE TEMP VIEW)**
```python
spark.sql('USE my_db_spark')
spark.sql('SHOW tables').show()
+-----------+---------+-----------+                                             
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|my_db_spark|   orders|      false|
+-----------+---------+-----------+
```

### Insert data from the `TempView` into the new `Table` (the persistent table)

In [ ]:
spark.sql("INSERT INTO orders \
            SELECT * \
            FROM orders_view")

In [ ]:
spark.sql("SELECT * FROM orders").show(10)

### Describe `Table`

In [ ]:
spark.sql("DESCRIBE TABLE orders").show()

In [ ]:
# For the persistent table 
spark.sql("DESCRIBE EXTENDED orders").show(truncate=False)              # Its a managed table

In [ ]:
# For the TempView
spark.sql("DESCRIBE EXTENDED orders_view").show()              

### Check the underline data in `HDFS` (Managed Table)

In [ ]:
%%bash

hadoop fs -ls hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse

In [ ]:
%%bash

hadoop fs -ls hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/orders

In [ ]:
%%bash 

hadoop fs -head hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/orders/part-00000-e687636c-e1bf-4205-baef-0315bae3cc48-c000

### Deleting the `table`

In [ ]:
spark.sql("DROP TABLE orders") # It actually deleted the metadata and the data (BOTH) 

In [ ]:
# spark.sql("DESCRIBE TABLE orders").show() # It will throw an ERROR 

In [ ]:
%%bash

hadoop fs -ls hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/orders

# It should give an error (Here we used MANAGED Table, and hence the data and metadata both got deleted when we ran DROP)


# Clean Up 

In [ ]:
spark.sql("DROP DATABASE my_db_spark")

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
spark.sql('USE default')

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
spark.sql("SHOW databases").show()

In [ ]:
%%bash

hadoop fs -ls hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db